# Creation of BlackBox Models for the Geotarget30 dataset

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# ADD OTHER FOLDERS TO THIS LIST TO ADD THEM TO THE sys.path
modules_to_add = [""]

this_file = os.path.abspath('')

for module in modules_to_add:
    p = Path(this_file).parent / module 
    if p.exists():
        sys.path.append(str(p))
        print(f"ADDED: {p}")
    else:
        print(f"ERROR: {p} doesn't exist")

ADDED: /home/gerardozinno/Desktop/Tesi/Code/mlem


In [3]:
print(sys.path)

['/home/gerardozinno/Desktop/Tesi/Code/mlem/notebooks', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python39.zip', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9/lib-dynload', '', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/IPython/extensions', '/home/gerardozinno/.ipython', '/home/gerardozinno/Desktop/Tesi/Code/mlem']


# CREATING THE TRAIN AND TEST DATASETS

In [4]:
WHOLE_DATASET_PATH = Path("../data/geotarget/geotarget_30.csv")
TRAIN_PATH = WHOLE_DATASET_PATH.parent / "train" / "train.csv"
TEST_PATH  = WHOLE_DATASET_PATH.parent / "test" / "test.csv"

In [5]:
TRAIN_SPLIT = .8
RAND_SEED   = 1234

In [6]:
if not (TRAIN_PATH.exists() and TEST_PATH.exists()):
    print(f"Couldn't find the train and/or test dataset(s) in:\n\t{TRAIN_PATH}\n\t{TEST_PATH}\n")
    if not WHOLE_DATASET_PATH.exists():
        print(f"ERROR: Couldn't even find {WHOLE_DATASET_PATH}")
        raise Exception("Can't find dataset")
    else:
        
        print(f"Creating train and test sets with a split of {TRAIN_SPLIT}% - {1-TRAIN_SPLIT:.2f}% and {RAND_SEED} as random seed")
        print('The dataset is split "as is", without preprocessing. The selection of the right columns is made by the respective Dataloader')
        df = pd.read_csv(WHOLE_DATASET_PATH)
        train, test = train_test_split(df, train_size=TRAIN_SPLIT, shuffle=True, random_state=RAND_SEED)
        TRAIN_PATH.parent.mkdir(exist_ok=True)
        TEST_PATH.parent.mkdir(exist_ok=True)
        train.to_csv(TRAIN_PATH, index=False)
        test.to_csv(TEST_PATH, index=False)
        print("train and test datasets created")

# CREATING THE MODELS

# LOADING THE DATA

In [7]:
from datasets.geotarget import Geotarget30
from torch.utils.data import DataLoader

In [8]:
train_set = Geotarget30(TRAIN_PATH)
test_set  = Geotarget30(TEST_PATH)

In [9]:
train_dataloader = DataLoader(train_set, batch_size=64)
test_dataloader  = DataLoader(test_set, batch_size=64)

# TRAINING LOOP

In [10]:
from blackboxes.pytorch.utilities import train

In [11]:
help(train)

Help on function train in module blackboxes.pytorch.utilities:

train(model: torch.nn.modules.module.Module, optimizer: torch.optim.optimizer.Optimizer, loss_fn: object, train_loader: torch.utils.data.dataloader.DataLoader, test_loader: torch.utils.data.dataloader.DataLoader, epochs: int = 20, device: str = 'cpu') -> None
    Training loop for training a model.
    
    Args:
        model: model to train.
        optimizer: optimizer to use.
        loss_fn: loss function.
        train_loader: DataLoader for loading the training data.
        test_loader: Dataloader for loading the test data
        epochs: Training epochs. (Default  = 20)
        device: Where to train the model, "cpu" or "cuda". (Default = "cpu")
    
    Returns:

